In [20]:
import pandas as pd

# 1) MG Salary

In [21]:
mg_salary = pd.DataFrame(pd.read_csv("MG Salary.csv"))

count = 0
for index, row in mg_salary.iterrows():
    if index > 200:
        break
    if row["Total Hours"] > 40:
        print(row["Name"] + " worked " + str(row["Total Hours"] - 40) + " overtime hours during the week of " + str(row["Week"]))
        count += 1

print()
print("Process complete. Detected " + str(count) + " employee(s) that worked overtime hours.")

Alexander Mazzella worked 5.0 overtime hours during the week of 7/31/22-8/6/22

Process complete. Detected 1 employee(s) that worked overtime hours.


In [22]:
ths_mg_salary = pd.DataFrame(pd.read_csv("ths_mg_salary.csv"))

count = 0
for index, row in ths_mg_salary.iterrows():
    if index > 200:
        break
    if row["Total Hours"] > 40:
        print(row["Name"] + " worked " + str(row["Total Hours"] - 40) + " overtime hours during the week of " + str(row["Week"]))
        count += 1

print()
print("Process complete. Detected " + str(count) + " employee(s) that worked overtime hours.")

Jack Tian worked 6.25 overtime hours during the week of 7/31/22-8/6/22
Rory Pan worked 7.25 overtime hours during the week of 7/31/22-8/6/22
Thomas Garity worked 7.0 overtime hours during the week of 7/31/22-8/6/22
Tomas Arevalo worked 1.0 overtime hours during the week of 8/7/22-8/13/22

Process complete. Detected 4 employee(s) that worked overtime hours.


# 2) Tutoring

In [23]:
tutoring = pd.DataFrame(pd.read_csv("tutoring.csv"))

In [24]:
def timeConverter(string):
    
    # function that converts string XX:XX XM --> integer in [0:86400]
    
    if not isinstance(string, str):
        return None
    
    res = 0
    
    if string[:2] == "12":
        res -= 43200
    
    j = 0
    
    hour = []
    while j < len(string):
        if string[j] == ":":
            j += 1
            break
        else:
            hour.append(string[j])
            j += 1
    
    minute = []
    while j < len(string):
        if string[j] == " ":
            j += 1
            break
        else:
            minute.append(string[j])
            j += 1
    
    res += int("".join(hour))*3600 + int("".join(minute))*60
    
    if string[j:] == "PM" or string[j:] == "pm":
        return res + 43200
    else:
        return res

In [25]:
from collections import defaultdict
from pprint import pprint

error_count = 0

# we will store employees in a custom data structure - dict of dicts, value is a list
employee_dict = defaultdict(lambda: defaultdict(list))

for index, row in tutoring.iterrows():
    # dealing with at most 300 employees — increase this number if more are hired
    if index > 300:
        break
    start, end = timeConverter(row["Start Time"]), timeConverter(row["End Time"])
    # edge case in which the interval starts before 12AM and ends after 12AM
    if isinstance(end, int) and isinstance(start, int) and end < start:
        end += 86400
    # use custom data structure to keep days separate for each employee, add intervals to each day
    employee_dict[row["Name"]][row["Date"]].append([start, end])
    
for employee in employee_dict:
    if isinstance(employee, float):
        nan = employee
    else:
        # sort intervals by start time
        for date in employee_dict[employee]:
            employee_dict[employee][date].sort()
    
del employee_dict[nan]

for employee in employee_dict:
    for date in employee_dict[employee]:
        intervals = employee_dict[employee][date]
        for j in range(len(intervals) - 1):
            if intervals[j][1] > intervals[j + 1][0]:
                print("OVERLAP  " + employee + " has an overlapping work interval logged on " + date)
                error_count += 1
        for start, end in intervals:
            if end - start > 14400:
                print("LONG     " + employee + " has a work interval lasting longer than 4 hours on " + date)
                error_count += 1
                
            if 0 <= start <= 21600:
                print("START    " + employee + " has a work interval starting between 12AM and 6AM on " + date)
                error_count += 1
            
print()
print("Process complete. Detected " + str(error_count) + " possible error(s).")

START    Abigail Chachkes has a work interval starting between 12AM and 6AM on 7/23/22
OVERLAP  Arjun Bhattarai has an overlapping work interval logged on 7/28/22
START    Chibuikem Uche has a work interval starting between 12AM and 6AM on 8/11/22
START    Emma Price has a work interval starting between 12AM and 6AM on 7/31/22
LONG     Harrison Wallace (Harrison) has a work interval lasting longer than 4 hours on 8/11/22
START    Nibrass Fathi has a work interval starting between 12AM and 6AM on 8/3/22
START    Nibrass Fathi has a work interval starting between 12AM and 6AM on 8/5/22
LONG     Nibrass Fathi has a work interval lasting longer than 4 hours on 8/7/22
START    Nibrass Fathi has a work interval starting between 12AM and 6AM on 8/7/22
START    Nibrass Fathi has a work interval starting between 12AM and 6AM on 8/10/22
START    Nibrass Fathi has a work interval starting between 12AM and 6AM on 8/12/22
START    David Hill (David Hill) has a work interval starting between 12AM an

# 3) Academies

In [26]:
aca = pd.DataFrame(pd.read_csv("aca.csv"))

error_count = 0

# we will store employees in a custom data structure - dict of dicts, value is a list
employee_dict = defaultdict(lambda: defaultdict(list))

for index, row in aca.iterrows():
    # dealing with at most 300 employees — increase this number if more are hired
    if index > 300:
        break
    start, end = timeConverter(row["start time"]), timeConverter(row["end time"])
    # edge case in which the interval starts before 12AM and ends after 12AM
    if isinstance(end, int) and isinstance(start, int) and end < start:
        end += 86400
    # use custom data structure to keep days separate for each employee, add intervals to each day
    employee_dict[row["employee"]][row["date"]].append([start, end])
    
for employee in employee_dict:
    if isinstance(employee, float):
        nan = employee
    else:
        # sort intervals by start time
        for date in employee_dict[employee]:
            employee_dict[employee][date].sort()
    
del employee_dict[nan]
if employee_dict["Totals"]:
    del employee_dict["Totals"]

for employee in employee_dict:
    for date in employee_dict[employee]:
        intervals = employee_dict[employee][date]
        for j in range(len(intervals) - 1):
            if intervals[j][1] > intervals[j + 1][0]:
                print("OVERLAP  " + employee + " has an overlapping work interval logged on " + date)
                error_count += 1
        for start, end in intervals:
            if end - start > 28800:
                print("LONG     " + employee + " has a work interval lasting longer than 8 hours on " + date)
                error_count += 1
                
            if 0 <= start <= 21600:
                print("START    " + employee + " has a work interval starting between 12AM and 6AM on " + date)
                error_count += 1
            
print()
print("Process complete. Detected " + str(error_count) + " possible error(s).")

START    Gabriel Khoury has a work interval starting between 12AM and 6AM on 8/4/22
START    Gabriel Khoury has a work interval starting between 12AM and 6AM on 8/11/22
START    Gabriel Khoury has a work interval starting between 12AM and 6AM on 8/12/22
START    Mya Joyce has a work interval starting between 12AM and 6AM on 8/7/22
START    William Flowers has a work interval starting between 12AM and 6AM on 7/31/22
START    William Flowers has a work interval starting between 12AM and 6AM on 8/6/22
START    William Flowers has a work interval starting between 12AM and 6AM on 8/7/22
START    Saswato Ray has a work interval starting between 12AM and 6AM on 8/6/22

Process complete. Detected 8 possible error(s).


# 4) Cleaners

In [27]:
cl = pd.DataFrame(pd.read_csv("cl.csv"))

error_count = 0

# we will store employees in a custom data structure - dict of dicts, value is a list
employee_dict = defaultdict(lambda: defaultdict(list))

for index, row in cl.iterrows():
    # dealing with at most 300 employees — increase this number if more are hired
    if index > 300:
        break
    start, end = timeConverter(row["start time"]), timeConverter(row["end time"])
    # edge case in which the interval starts before 12AM and ends after 12AM
    if isinstance(end, int) and isinstance(start, int) and end < start:
        end += 86400
    # use custom data structure to keep days separate for each employee, add intervals to each day
    employee_dict[row["employee"]][row["date"]].append([start, end])
    
for employee in employee_dict:
    if isinstance(employee, float):
        nan = employee
    else:
        # sort intervals by start time
        for date in employee_dict[employee]:
            employee_dict[employee][date].sort()
    
del employee_dict[nan]

for employee in employee_dict:
    for date in employee_dict[employee]:
        intervals = employee_dict[employee][date]
        for j in range(len(intervals) - 1):
            if intervals[j][1] > intervals[j + 1][0]:
                print("OVERLAP  " + employee + " has an overlapping work interval logged on " + date)
                error_count += 1
        for start, end in intervals:
            if end - start > 28800:
                print("LONG     " + employee + " has a work interval lasting longer than 8 hours on " + date)
                error_count += 1
                
            if 0 <= start <= 21600:
                print("START    " + employee + " has a work interval starting between 12AM and 6AM on " + date)
                error_count += 1
            
print()
print("Process complete. Detected " + str(error_count) + " possible error(s).")


Process complete. Detected 0 possible error(s).


# 5) The Harvard Shop

In [69]:
import datetime

def isSunday(string):
    
    # takes a date MM/DD/YY --> returns True if Sunday, else False
    
    if not isinstance(string, str):
        return False
    
    month, day, year = [int(x) for x in string.split('/')]
    weekday = datetime.date(year, month, day).weekday()
    
    # 6 means the day is a Sunday
    
    return weekday == 6

In [72]:
ths = pd.DataFrame(pd.read_csv("ths.csv"))

error_count = 0

# we will store employees in a custom data structure - dict of dicts, value is a list
employee_dict = defaultdict(lambda: defaultdict(list))

for index, row in ths.iterrows():
    # dealing with at most 500 employees — increase this number if more are hired
    if index > 500:
        break
    start, end = timeConverter(row["start time"]), timeConverter(row["end time"])
    # edge case in which the interval starts before 12AM and ends after 12AM
    if isinstance(end, int) and isinstance(start, int) and end < start:
        end += 86400
    # use custom data structure to keep days separate for each employee, add intervals to each day
    employee_dict[row["employee"]][row["date"]].append([start, end])
    
    # THS Specific — check isSunday() and ensure OT hours if so, and reverse
    if isSunday(row["date"]):
        if int(row["regular"]) > 0:
            print("SUNDAY   " + row["employee"] + " logged regular hours on Sunday, " + row["date"])
            error_count += 1
    else:
        if isinstance(row["date"], str) and int(row["special"]) > 0:
            print("SPECIAL  " + row["employee"] + " logged special hours on Mon-Sat, " + row["date"])
            error_count += 1
    
for employee in employee_dict:
    if isinstance(employee, float):
        nan = employee
    else:
        # sort intervals by start time
        for date in employee_dict[employee]:
            employee_dict[employee][date].sort()
    
del employee_dict[nan]
if employee_dict["Totals"]:
  del employee_dict["Totals"]

for employee in employee_dict:
    for date in employee_dict[employee]:
        intervals = employee_dict[employee][date]
        for j in range(len(intervals) - 1):
            if intervals[j][1] > intervals[j + 1][0]:
                print("OVERLAP  " + employee + " has an overlapping work interval logged on " + date)
                error_count += 1
        for start, end in intervals:
            if end - start > 28800:
                print("LONG     " + employee + " has a work interval lasting longer than 8 hours on " + date)
                error_count += 1
                
            if 0 <= start <= 21600:
                print("START    " + employee + " has a work interval starting between 12AM and 6AM on " + date)
                error_count += 1
            
print()
print("Process complete. Detected " + str(error_count) + " possible error(s).")

SPECIAL  Indi Wagner logged special hours on Mon-Sat, 7/28/22
SUNDAY   Chisimdi Aguwa logged regular hours on Sunday, 7/31/22
LONG     Amechi Egbunike has a work interval lasting longer than 8 hours on 8/2/22
LONG     Audrey Propp has a work interval lasting longer than 8 hours on 8/8/22
LONG     Ava Mulrooney has a work interval lasting longer than 8 hours on 8/8/22
LONG     Kaya Vadhan has a work interval lasting longer than 8 hours on 8/3/22
LONG     Kaya Vadhan has a work interval lasting longer than 8 hours on 8/5/22
LONG     Kara Xie has a work interval lasting longer than 8 hours on 8/9/22
LONG     Kimberly Baptiste has a work interval lasting longer than 8 hours on 8/2/22
LONG     Kimberly Baptiste has a work interval lasting longer than 8 hours on 8/3/22
LONG     Kimberly Baptiste has a work interval lasting longer than 8 hours on 8/4/22
LONG     Kimberly Baptiste has a work interval lasting longer than 8 hours on 8/5/22
LONG     Kimberly Baptiste has a work interval lasting l

# 6) Trademark Tours

In [77]:
tt = pd.DataFrame(pd.read_csv("tt.csv"))

position_map = {
    "Ambassador": [16, 18, 20],
    "Guide-in-Training": [14.25],
    "Harvard Private Guide": [22],
    "Harvard Public Guide": [15],
    "HBS Guide": [22],
    "Kiosk": [15],
    "MIT Private Guide": [22],
    "MIT Public Guide": [15]
}

error_count = 0

# we will store employees in a custom data structure - dict of dicts, value is a list
employee_dict = defaultdict(lambda: defaultdict(list))

for index, row in tt.iterrows():
    # dealing with at most 500 employees — increase this number if more are hired
    if index > 400:
        break
    start, end = timeConverter(row["start time"]), timeConverter(row["end time"])
    # edge case in which the interval starts before 12AM and ends after 12AM
    if isinstance(end, int) and isinstance(start, int) and end < start:
        end += 86400
    # use custom data structure to keep days separate for each employee, add intervals to each day
    employee_dict[row["employee"]][row["date"]].append([start, end])
    
    # TT Specific — check if the rate is correct for each employee
    if row["Position"] in position_map:
        if float(row["Rate"]) not in position_map[row["Position"]]:
            print("RATE     " + row["employee"] + " has an incorrect position rate paid on " + row["date"])
            error_count += 1
    
for employee in employee_dict:
    if isinstance(employee, float):
        nan = employee
    else:
        # sort intervals by start time
        for date in employee_dict[employee]:
            employee_dict[employee][date].sort()
    
del employee_dict[nan]
if employee_dict["Totals"]:
  del employee_dict["Totals"]

for employee in employee_dict:
    for date in employee_dict[employee]:
        intervals = employee_dict[employee][date]
        for j in range(len(intervals) - 1):
            if intervals[j][1] > intervals[j + 1][0]:
                print("OVERLAP  " + employee + " has an overlapping work interval logged on " + date)
                error_count += 1
        for start, end in intervals:
            if end - start > 28800:
                print("LONG     " + employee + " has a work interval lasting longer than 8 hours on " + date)
                error_count += 1
                
            if 0 <= start <= 21600:
                print("START    " + employee + " has a work interval starting between 12AM and 6AM on " + date)
                error_count += 1
            
print()
print("Process complete. Detected " + str(error_count) + " possible error(s).")

RATE     Kat Boit has an incorrect position rate paid on 31-Jul
RATE     Marcus Knoke has an incorrect position rate paid on 31-Jul
OVERLAP  Marcus Knoke has an overlapping work interval logged on 31-Jul
OVERLAP  Lillie Cooper has an overlapping work interval logged on 9-Aug
START    Lillie Cooper has a work interval starting between 12AM and 6AM on 12-Aug

Process complete. Detected 5 possible error(s).
